# **Cubic Crystal Structure**

[Go back to index](./index.ipynb)

<hr style="height:1px;border:none;color:#cccccc;background-color:#cccccc;" />

<p style="text-align: justify;font-size:15px;width:90%">  
For molecular structure, the cubic crystal system has three different crystal structures, namely the face centre cubic (FCC), body center cubic (BCC) and simple cubic structures.
</p>

In [ ]:
from pythreejs import *
import numpy as np
from IPython.display import display
from ipywidgets import HTML, Text, Output, VBox
from traitlets import link, dlink
import math

In [ ]:
vertices = [
    [0, 0, 0],
    [0, 0, 1],
    [0, 1, 0],
    [0, 1, 1],
    [1, 0, 0],
    [1, 0, 1],
    [1, 1, 0],
    [1, 1, 1]
]

faces = [
    [0, 1, 3],
    [0, 3, 2],
    [0, 2, 4],
    [2, 6, 4],
    [0, 4, 1],
    [1, 4, 5],
    [2, 3, 6],
    [3, 7, 6],
    [1, 5, 3],
    [3, 5, 7],
    [4, 6, 5],
    [5, 6, 7]
]

surfaces = [
    [0, 0.5, 0.5],
    [1, 0.5, 0.5],
    [0.5, 0.5, 0],
    [0.5, 0.5, 1],
    [0.5, 0, 0.5],
    [0.5, 1, 0.5]
]

vertexcolors = ['#000000', '#0000ff', '#00ff00', '#ff0000',
                '#00ffff', '#ff00ff', '#ffff00', '#ffffff']

# Map the vertex colors into the 'color' slot of the faces
faces = [f + [None, [vertexcolors[i] for i in f], None] for f in faces]

# balls at vertex 
balls = [Mesh(geometry=SphereGeometry(radius=0.1, widthSegments=32, heightSegments=32,
                                     phiStart = 0, phiLength = math.pi*2,
                                     thetaStart = 0, thetaLength = math.pi,), 
            material=MeshLambertMaterial(color='red', opacity = 0.7, transparent = True),
            position=i) for i in vertices]

cball = Mesh(geometry=SphereGeometry(radius=0.1, widthSegments=32, heightSegments=32,
                                     phiStart = 0, phiLength = math.pi*2,
                                     thetaStart = 0, thetaLength = math.pi), 
            material=MeshLambertMaterial(color='gold'),
            position=[0.5, 0.5, 0.5])

sballs = [Mesh(geometry=SphereGeometry(radius=0.1, widthSegments=32, heightSegments=32,
                                     phiStart = 0, phiLength = math.pi*2,
                                     thetaStart = 0, thetaLength = math.pi), 
            material=MeshLambertMaterial(color='red', opacity = 0.7, transparent = True),
            position=i) for i in surfaces]

# Create the geometry:
cubeGeometry = Geometry(vertices=vertices,
    faces=faces,
    colors=vertexcolors,)
# Calculate normals per face, for nice crisp edges:
cubeGeometry.exec_three_obj_method('computeFaceNormals')

box = BoxGeometry(1,1,1)
edgesGemoetry = EdgesGeometry(box);

# Create a mesh. Note that the material need to be told to use the vertex colors.
myobjectCube = Mesh(
    geometry=box,
    material= MeshPhongMaterial(color = 'blue', opacity = 0.4, transparent = True),
    position=[0.5, 0.5, 0.5],   # Center the cube
)


edges = LineSegments(
    geometry = edgesGemoetry,
    material = LineBasicMaterial(color = 'black', linewidth = 3),
    position = [0.5, 0.5, 0.5]
)


# Set up a scene and render it:
cCube = PerspectiveCamera(position=[3, 3, 3], fov=20,
                      children=[DirectionalLight(color='#ffffff', position=[-3, 5, 1], intensity=0.5)])


sceneCube = Scene(children= balls + sballs + [myobjectCube, cCube, cball, edges, AmbientLight(color='#dddddd')])

rendererCube = Renderer(camera=cCube, background='black', background_opacity=1,
                        scene=sceneCube, controls=[OrbitControls(controlling=cCube)], 
                        width=500, height=500)

In [ ]:
from ipywidgets import HBox, VBox, Button, RadioButtons, FloatSlider 

def on_value_change(change):
    if structureSelection.value == 'Simple Cubic':
        cball.visible = False
        for i in sballs:
            i.visible = False
    elif structureSelection.value == 'BCC':
        cball.visible = True
        for i in sballs:
            i.visible = False
    else:
        cball.visible = False
        for i in sballs:
            i.visible = True

def on_radius_change(change):
    for i in balls + sballs:
        i.scale = [10*radius.value, 10*radius.value, 10*radius.value]
        
    cball.scale = [10*radius.value, 10*radius.value, 10*radius.value]
    
    
structureSelection = RadioButtons(
        options = ['FCC', 'BCC', 'Simple Cubic'],
        description = 'Structures',
        disabled = False)

structureSelection.observe(on_value_change, names='value')

radius = FloatSlider(
    value=0.0,
    min=0.0,
    max=0.5,
    step=0.1,
    description='Radius (a):',
    disabled=False,
    continuous_update=False,
    orientation='horizontal',
    readout=True,
    readout_format='.1f',
)

radius.observe(on_radius_change, names = 'value')

display(VBox([structureSelection, rendererCube, radius]))